<a href="https://colab.research.google.com/github/saloniupadhyay2003/spaCy_tutorial/blob/main/Resume_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RESUME PARSER USING SPACY

Data:


*   https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git




Installations

In [ ]:
!pip install spacy_transformers
!pip install -U spacy

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
!nvidia-smi

Tue Jul 25 18:40:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

fatal: destination path 'CV-Parsing-using-Spacy-3' already exists and is not an empty directory.


In [ ]:
import json

IMPORTING DATA

In [ ]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [ ]:
len(cv_data)

200

In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg


2023-07-25 19:38:25.023120: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db


TRAINING MODEL

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size = 0.3)

In [ ]:
len(test)

60

In [ ]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 82.51it/s]


In [ ]:
len(db.tokens)

60

In [ ]:
!python -m spacy train /content/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-07-25 19:39:52.884876: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-07-25 19:40:01,968] [INFO] Set up nlp object from config
[2023-07-25 19:40:01,985] [INFO] Pipeline: ['transformer', 'ner']
[2023-07-25 19:40:01,989] [INFO] Created vocabulary
[2023-07-25 19:40:01,989] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Ro

#Model test

In [ ]:
nlp = spacy.load('/content/output/model-best')

In [ ]:
doc = nlp('my name is Laxmi Kant , I worked at Amazon. I have 2 years of experience')
for ent in doc.ents:
  print(ent.text, "   ->>>>>>>> ", ent.label_)

Laxmi Kant    ->>>>>>>>  Name
Amazon    ->>>>>>>>  Companies worked at
2 years    ->>>>>>>>  Years of Experience


In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.7 MB/s eta 0:00:00


In [ ]:
import sys, fitz

In [ ]:
fname = '/content/CV-Parsing-using-Spacy-3/data/test/Smith Resume.pdf'

In [ ]:
doc = fitz.open(fname)

In [ ]:
doc

Document('/content/CV-Parsing-using-Spacy-3/data/test/Smith Resume.pdf')

In [ ]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [ ]:
text

' Michael Smith \nBI / Big Data/ Azure \nManchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f \n \n10+ years of Experience in Designing, Development, Administration, Analysis, \nManagement \ninthe \nBusiness \nIntelligence \nData \nwarehousing, \nClient \nServer \nTechnologies, Web-based Applications, cloud solutions and Databases. \nData warehouse: Data analysis, star/ snow flake schema data modeling and design \nspecific todata warehousing and business intelligence environment. \nDatabase: Experience in database designing, scalability, back-up and recovery, \nwriting andoptimizing SQL code and Stored Procedures, creating functions, views, \ntriggers and indexes.  \nCloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL \nAzure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure \ndata lake analytics(U-SQL). \nBig Data: Worked Azure data lake store/analytics for big data processing and Azure \ndata factoryto schedule U

In [ ]:
text = text.strip()

In [ ]:
text = " ".join(text.split())

In [ ]:
text

'Michael Smith BI / Big Data/ Azure Manchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f 10+ years of Experience in Designing, Development, Administration, Analysis, Management inthe Business Intelligence Data warehousing, Client Server Technologies, Web-based Applications, cloud solutions and Databases. Data warehouse: Data analysis, star/ snow flake schema data modeling and design specific todata warehousing and business intelligence environment. Database: Experience in database designing, scalability, back-up and recovery, writing andoptimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL). Big Data: Worked Azure data lake store/analytics for big data processing and Azure data factoryto schedule U-SQL jobs. Designed and developed end to end big 

In [ ]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, "   ->>>>>>>> ", ent.label_)

Michael Smith    ->>>>>>>>  Name
BI    ->>>>>>>>  Degree
Manchester    ->>>>>>>>  Location
indeed.com/r/falicent/140749dace5dc26f    ->>>>>>>>  Email Address
10+    ->>>>>>>>  Years of Experience
The University of Manchester    ->>>>>>>>  College Name
2007    ->>>>>>>>  Graduation Year
problem solving (Less than 1 year), project lifecycle (Less than 1 year), project manager (Less than 1 year), technical assistance. (Less than 1 year) ADDITIONAL INFORMATION Professional Skills Excellent analytical, problem solving, communication, knowledge transfer and interpersonalskills with ability to interact with individuals at all the levels Quick learner and maintains cordial relationship with project manager and team members andgood performer both in team and independent job environments Positive attitude towards superiors &amp; peers Supervised junior developers throughout project lifecycle and provided technical assistance.    ->>>>>>>>  Skills
